# Иллюстрация  разбиения датасета Amazon на train/test/split
<blockquote>
    <p>Предобработки признаков для датасета Amazon, используемая здесь вынесена в 
       <a href="https://github.com/AlgoMathITMO/sber-simulator/blob/experiments-new/experiments/Amazon/amazon.py" title="amazon.py">файл</a>,
       иллюстрация её работы продемонстрирована в <a href="https://github.com/AlgoMathITMO/sber-simulator/blob/experiments-new/experiments/Amazon/amazon_processing.ipynb" 
       title="amazon_preprocessing">ноутбуке</a>.</p>
</blockquote>

### $\textbf{Содержание}$:

### $\textbf{I. Загрузка данных }$
#### - Чтение данных с диска;
---

### $\textbf{II. Разбиение данных для эксперимента}$
### Для разбиения данных на $\it{train/test/split}$ производится деление исходного датасета *df_rating* по квантилям атрибута $\it{timestamp}$, $\mathbb{q}$ для генерации признаков:
#### $\it{rating}_{t}$ = *df_rating*$[0, \mathbb{q}_{t}]$, где $\mathbb{q}_{train}=0.5$, $\mathbb{q}_{val}=0.75$, $\mathbb{q}_{test}=1$:
#### - $\it{rating}_{train}$ = *df_rating*$[0, 0.5]$;
#### - $\it{rating}_{val}$ = *df_rating*$[0, 0.75]$;
#### - $\it{rating}_{test}$ = *df_rating*$[0, 1]$;
### Далее для каждого из промежутков {$\it{rating}_{train}$, $\it{rating}_{val}$, $\it{rating}_{test}$} генерируются соответствующие им признаки пользователей и предложений:
#### - $\it{items}_{t}$, $\it{users}_{t}$, $\it{rating}_{t}$ = data_processing(movies, $\it{rating}_{t}$, tags), $t \in \{\it{train}, \it{val}, \it{test}\}$;
### После чего формируются окончательные рейтинги:
#### - $\it{rating}_{train}$ = $\it{rating}_{train}$ = *df_rating*$[0, 0.5]$;
#### - $\it{rating}_{val}$ = $\it{rating}_{val}$[$\mathbb{q}>\mathbb{q}_{train}$] = *df_rating*$(0.5, 0.75]$;
#### - $\it{rating}_{test}$ = $\it{rating}_{test}$[$\mathbb{q}>\mathbb{q}_{val}$] = *df_rating*$(0.75, 1]$;

<blockquote>
    <p>То есть, если для генерации признаков для валидационного набора данных мы используем временные метки с 0 по 0.75 квантиль, то в качестве рейтингов мы возьмем оценки
       только с 0.5 по 0.75 квантили. Аналогично для тестового набора: все временные метки для генерации признаков, но в качестве рейтингов только оценки с 0.75 по 1
       квантили.</p>
</blockquote>
<hr>

In [ ]:
import pandas as pd
import numpy as np
import re
import itertools
import tqdm

from amazon import data_processing, get_spark_session

[nltk_data] Downloading package punkt to
[nltk_data]     /data/home/agurov/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /data/home/agurov/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     /data/home/agurov/nltk_data...
[nltk_data]   Package words is already up-to-date!
/data/home/agurov/.conda/envs/sber3.8/lib/python3.8/site-packages/pyspark/sql/pandas/functions.py:394: UserWarning: In Python 3.6+ and Spark 3.0+, it is preferred to specify type hints for pandas UDF instead of specifying pandas UDF type which will be deprecated in the future releases. See SPARK-28264 for more details.
  warnings.warn(


In [ ]:
file_name_parquet = "all.parquet"
save_path = "hdfs://namenode:9000/Sber_data/Amazon/final_data"

spark = get_spark_session(1)

I. Загрузка данных

In [ ]:
df = pd.read_parquet(file_name_parquet)
df_sp = spark.createDataFrame(df)

### II. Разбиение данных для эксперимента

### Разбиение df_rating на train/test/validation части по квантилям timestamp:
####  - train [0, 0.5]
####  - validation [0, 0.75]
####  - test [0, 1.]

In [3]:
q_50, q_75 = df_sp.approxQuantile("timestamp", [0.5, 0.75], 0)
print(f"Quantile: 0.5 - {q_50}, 0.75 - {q_75}")

Quantile: 0.5 - 1189555200.0, 0.75 - 1301443200.0


In [ ]:
df_train = df_procc.filter(sf.col("timestamp") <= q_50)
df_train.cache()

In [ ]:
df_val = df_procc.filter(sf.col("timestamp") <= q_75)
df_val.cache()

In [ ]:
df_test = df_procc
df_test.cache()

#### Генерация признаков по временным промежуткам

#### Train data

In [ ]:
df_users_train, df_items_train, df_rating_train = data_processing(df_train)

df_items_train.write.parquet(os.path.join(save_path, r'train/items.parquet'))
df_users_train.write.parquet(os.path.join(save_path, r'train/users.parquet'))
df_rating_train.write.parquet(os.path.join(save_path, r'train/rating.parquet'))

df_train.unpersist()

#### Validation data

In [ ]:
df_users_val, df_items_val, df_rating_val = data_processing(df_val)

df_rating_val = df_rating_val.filter(sf.col("timestamp") > q_50)
df_items_val.write.parquet(os.path.join(save_path, r'val/items.parquet'))
df_users_val.write.parquet(os.path.join(save_path, r'val/users.parquet'))
df_rating_val.write.parquet(os.path.join(save_path, r'val/rating.parquet'))

df_val.unpersist()

#### Test data

In [ ]:
df_users_test, df_items_test, df_rating_test = data_processing(df_test)

df_rating_test = df_rating_test.filter(sf.col("timestamp") > q_75)
df_items_test.write.parquet(os.path.join(save_path, r'val/items.parquet'))
df_users_test.write.parquet(os.path.join(save_path, r'val/users.parquet'))
df_rating_test.write.parquet(os.path.join(save_path, r'val/rating.parquet'))

df_test.unpersist()